In [70]:
import sqlite3
import datetime as datetime
conn = sqlite3.connect("testDB.db")

In [9]:
cur = conn.cursor()

In [4]:
import os
import itertools
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
path = 'C:/Users/Pragati Shah/subset/'

In [96]:
#This function takes a directory path, parses all the xml files present in the directory, and returns the key attributes in
#the format of a dataframe.


import os
def articleToDF(path):
    mainDF=pd.DataFrame(columns=["doi","pmid","journal","publisher_id","publisher_name","authors","editors","subjects","title","alt_title","date_pub","date_rec","abstract","body"])
    tempDF=pd.DataFrame()
    for fn in os.listdir(path):
        author_surnames=[]
        author_names=[]
        editor_surnames=[]
        editor_names=[]
        subjects=[]
        title=[]
        alt_title=[]
        authors=[]
        editors=[]
        abstract=" "
        body=" "
        journal=" "
        publisher_id=" "
        publisher_name=" "
        rday=0
        rmonth=0
        ryear=0
        pday=0
        pmonth=0
        pyear=0
        tree = ET.parse(path+fn)  
        root = tree.getroot() 
        for elem in tree.iter(tag='article-id'):
            a=str(elem.attrib)
            if (a.find('pmid')>-1):        
                pmid=elem.text
            if (a.find('doi')>-1):        
                doi=elem.text
        for elem in tree.iter(tag='contrib'):
            a=str(elem.attrib)
            if (a.find('author')>-1):
                for i in elem.iter(tag='surname'):
                    #print (i.tag, i.attrib,i.text)
                    author_surnames.append(i.text)
                for i in elem.iter(tag='given-names'):
                    #print (i.tag, i.attrib,i.text)
                    author_names.append(i.text)
            spaces=' '* len(author_names)
            authors=list(map(operator.add,list(map(operator.add, author_names,spaces)), author_surnames))
            if (a.find('editor')>-1):
                for i in elem.iter(tag='surname'):
                    #print (i.tag, i.attrib,i.text)
                    editor_surnames.append(i.text)
                for i in elem.iter(tag='given-names'):
                    #print (i.tag, i.attrib,i.text)
                    editor_names.append(i.text)
            spaces=' '* len(editor_names)
            editors=list(map(operator.add,list(map(operator.add, editor_names,spaces)), editor_surnames))
                    
        for a in root.iter('article-meta'):
            for elem in a.iter('article-title'):
                title.append(elem.text)
            for elem in a.iter('alt-title'):
                alt_title.append(elem.text)
            for elem in a.iter('abstract'):    
                b=ET.tostring(elem, method='text', encoding='unicode')
                abstract=b
         
        for subject in root.iter('subject'):
            subjects.append(subject.text)
            
        for a in root.iter('pub-date'):
            for elem in a.iter('day'):
                pday=elem.text
            for elem in a.iter('month'):
                pmonth=elem.text
            for elem in a.iter('year'):
                #print(elem.text)
                pyear=elem.text
        str_pdate=str(pyear)+'-'+str(pmonth)+'-'+str(pday)
        date_pub=datetime.datetime.strptime(str_pdate, "%Y-%m-%d")

        for a in root.iter('date'):
            x=str(a.attrib)
            #print(a.find('author'))
            if (x.find('received')>-1):
                for elem in a.iter(tag='day'):
                    rday=elem.text
                for elem in a.iter(tag='month'):
                    rmonth=elem.text
                for elem in a.iter(tag='year'):
                    #print(elem.text)
                    ryear=elem.text
        str_rdate=str(ryear)+'-'+str(rmonth)+'-'+str(rday)
        date_rec=datetime.datetime.strptime(str_rdate, "%Y-%m-%d")
        
        for a in root.iter('body'):
            b=ET.tostring(a, method='text', encoding='unicode')
            body=b
        
        for elem in tree.iter(tag='journal-title'):
            journal=elem.text
            
        for elem in tree.iter(tag='journal-id'):
            a=str(elem.attrib)
            if (a.find('publisher-id')>-1):
                publisher_id=elem.text
             
        for elem in tree.iter(tag='journal-meta'):    
            for elem1 in elem.iter(tag='publisher-name'):    
                publisher_name=elem1.text

        mainDF.loc[len(mainDF)]=[doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body]
    return(mainDF)


In [97]:
DF=articleToDF(path)
DF.head()

,doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body
0,10.1371/journal.pone.0000100,17183633,PLoS ONE,plos,Public Library of Science,"[Justin A. Harris, Ehsan Arabzadeh, Adrienne L...",[Hernan Lopez-Schier],"[Research Article, Neuroscience/Sensory Systems]",[Factors Affecting Frequency Discrimination of...,[Vibrotactile Discrimination],2006-12-20,2006-10-19,BackgroundMeasuring perceptual judgments about...,IntroductionMany investigations of sensory pro...
1,10.1371/journal.pone.0000008,17183712,PLoS ONE,plos,Public Library of Science,"[Maxim V. Kapralov, Dmitry A. Filatov]",[Konrad Scheffler],"[Research Article, Evolutionary Biology]",[Molecular Adaptation during Adaptive Radiatio...,[None],2006-12-20,2006-08-07,Background“Explosive” adaptive radiations on i...,IntroductionThe most dramatic “bursts” of adap...
2,10.1371/journal.pone.0000061,17183692,PLoS ONE,plos,Public Library of Science,"[Andrey Rzhetsky, Tian Zheng, Chani Weinreb]",[Nick Monk],"[Research Article, Cell Biology, Computational...",[Self-Correcting Maps of Molecular Pathways],[Self-Correcting Pathways],2006-12-20,2006-09-15,Reliable and comprehensive maps of molecular p...,IntroductionScientific innovation often procee...
3,10.1371/journal.pone.0000094,17183726,PLoS ONE,plos,Public Library of Science,"[Jingchun Zhu, Ashwini Jambhekar, Aaron Sarver...",[Gustavo Stolovitzky],"[Research Article, Genetics and Genomics/Funct...",[A Bayesian Network Driven Approach to Model t...,[Nitric Oxide Response Network],2006-12-20,2006-09-16,The transcriptional response to exogenously su...,IntroductionNitric oxide (NO·) is a critical m...
4,10.1371/journal.pone.0000011,17183637,PLoS ONE,plos,Public Library of Science,"[Matthew E. Falagas, Konstantinos N. Fragoulis...",[Bernhard Baune],"[Research Article, Public Health and Epidemiol...",[A Comparative Study on the Cost of New Antibi...,[Cost of New Drugs],2006-12-20,2006-08-28,BackgroundDrug treatment is becoming more expe...,IntroductionThe investment of societies around...


In [98]:
DF.dtypes

doi                       object
pmid                      object
journal                   object
publisher_id              object
publisher_name            object
authors                   object
editors                   object
subjects                  object
title                     object
alt_title                 object
date_pub          datetime64[ns]
date_rec          datetime64[ns]
abstract                  object
body                      object
dtype: object

In [103]:
cols=[i for i in DF.columns if i not in ["date_pub","date_rec"]]
for col in cols:
    DF[col]=DF[col].astype('str')

In [104]:
DF.dtypes

doi                       object
pmid                      object
journal                   object
publisher_id              object
publisher_name            object
authors                   object
editors                   object
subjects                  object
title                     object
alt_title                 object
date_pub          datetime64[ns]
date_rec          datetime64[ns]
abstract                  object
body                      object
dtype: object

In [105]:
DF.to_sql("trial1", conn, if_exists="replace")

In [106]:
pd.read_sql_query("select * from trial1;", conn)

,index,doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body
0,0,10.1371/journal.pone.0000100,17183633,PLoS ONE,plos,Public Library of Science,"['Justin A. Harris', 'Ehsan Arabzadeh', 'Adrie...",['Hernan Lopez-Schier'],"['Research Article', 'Neuroscience/Sensory Sys...",['Factors Affecting Frequency Discrimination o...,['Vibrotactile Discrimination'],2006-12-20 00:00:00,2006-10-19 00:00:00,BackgroundMeasuring perceptual judgments about...,IntroductionMany investigations of sensory pro...
1,1,10.1371/journal.pone.0000008,17183712,PLoS ONE,plos,Public Library of Science,"['Maxim V. Kapralov', 'Dmitry A. Filatov']",['Konrad Scheffler'],"['Research Article', 'Evolutionary Biology']",['Molecular Adaptation during Adaptive Radiati...,[None],2006-12-20 00:00:00,2006-08-07 00:00:00,Background“Explosive” adaptive radiations on i...,IntroductionThe most dramatic “bursts” of adap...
2,2,10.1371/journal.pone.0000061,17183692,PLoS ONE,plos,Public Library of Science,"['Andrey Rzhetsky', 'Tian Zheng', 'Chani Weinr...",['Nick Monk'],"['Research Article', 'Cell Biology', 'Computat...",['Self-Correcting Maps of Molecular Pathways'],['Self-Correcting Pathways'],2006-12-20 00:00:00,2006-09-15 00:00:00,Reliable and comprehensive maps of molecular p...,IntroductionScientific innovation often procee...
3,3,10.1371/journal.pone.0000094,17183726,PLoS ONE,plos,Public Library of Science,"['Jingchun Zhu', 'Ashwini Jambhekar', 'Aaron S...",['Gustavo Stolovitzky'],"['Research Article', 'Genetics and Genomics/Fu...",['A Bayesian Network Driven Approach to Model ...,['Nitric Oxide Response Network'],2006-12-20 00:00:00,2006-09-16 00:00:00,The transcriptional response to exogenously su...,IntroductionNitric oxide (NO·) is a critical m...
4,4,10.1371/journal.pone.0000011,17183637,PLoS ONE,plos,Public Library of Science,"['Matthew E. Falagas', 'Konstantinos N. Fragou...",['Bernhard Baune'],"['Research Article', 'Public Health and Epidem...",['A Comparative Study on the Cost of New Antib...,['Cost of New Drugs'],2006-12-20 00:00:00,2006-08-28 00:00:00,BackgroundDrug treatment is becoming more expe...,IntroductionThe investment of societies around...
5,5,10.1371/journal.pone.0000131,17205135,PLoS ONE,plos,Public Library of Science,"['Sara O. Dean', 'James A. Spudich']",['Robert Insall'],"['Research Article', 'Cell Biology']","[""Rho Kinase's Role in Myosin Recruitment to t...",['Rho Kinase in Cytokinesis'],2006-12-27 00:00:00,2006-10-16 00:00:00,BackgroundMyosin II recruitment to the equator...,IntroductionCytokinesis involves the formation...
6,6,10.1371/journal.pone.0000077,17183709,PLoS ONE,plos,Public Library of Science,"['Samantha Kerry', 'Michael TeKippe', 'Nathan ...",['Eleftherios Mylonakis'],"['Research Article', 'Immunology/Innate Immuni...",['GATA Transcription Factor Required for Immun...,['ELT-2 in Innate Immunity'],2006-12-20 00:00:00,2006-11-09 00:00:00,"In the past decade, Caenorhabditis elegans has...",IntroductionThe study of innate immunity recei...
7,7,10.1371/journal.pone.0000027,17183654,PLoS ONE,plos,Public Library of Science,"['Colin A. Russell', 'Leslie A. Real', 'David ...",['Matthew Baylis'],"['Research Article', 'Ecology', 'Infectious Di...",['Spatial Control of Rabies on Heterogeneous L...,['Spatial Control of Rabies'],2006-12-20 00:00:00,2006-09-21 00:00:00,Rabies control in terrestrial wildlife reservo...,IntroductionAn emerging paradigm for the sprea...
8,8,10.1371/journal.pone.0000080,17183713,PLoS ONE,plos,Public Library of Science,"['Zhihua Jiang', 'Jennifer J. Michal', 'Galen ...",['Neil Hall'],"['Research Article', 'Genetics and Genomics/An...",['Cross Species Association Examination of UCN...,['UCN3 and CRHR2 for Antiobesity'],2006-12-20 00:00:00,2006-10-07 00:00:00,BackgroundObesity now constitutes a leading gl...,IntroductionObesity has increased at a fast ra...
9,9,10.1371/journal.pone.0000102,17183635,PLoS ONE,plos,Public Library of Science,"['Fatema A. Legra